<a href="https://colab.research.google.com/github/developerabhi14/ML-Notebooks/blob/main/ChatGPT_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-03-23 07:51:45--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2025-03-23 07:51:45 (17.0 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [ ]:
with open("input.txt", "r", encoding= "utf-8") as f:
  text=f.read()

In [ ]:
print(len(text))

1115394


In [ ]:
text[:1000]

"First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this in hunger 

In [ ]:
chars=sorted(list(set(text)))
vocab_size=len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [ ]:
# Create a mapping frm characters to integers
stoi={ ch:i for i, ch in enumerate(chars) }
itos={ i:ch for i, ch in enumerate(chars) }
encode=lambda s: [stoi[c] for c in s] #encoder takes a string, outputs a list of integers
decode=lambda l: ''.join([itos[i] for i in l]) # decoder: takes a list of integers, outputs a string

print(encode("hello"))
print(decode(encode("hello")))

[46, 43, 50, 50, 53]
hello


In [ ]:
import torch

data=torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.type)
print(data[:1000])

torch.Size([1115394]) <built-in method type of Tensor object at 0x78ee5267d9d0>
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 

In [ ]:
#split into test and train
n=int(0.9*len(data)) # first 90% will be train, rest will be val
train_data=data[:n]
val_data=data[n:]

In [ ]:
block_size=8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [ ]:
x=train_data[:block_size]
y=train_data[1:block_size+1]
for t in range(block_size):
  context=x[:t+1]
  target=y[t]
  print(f"when input is {context} target is {target}")

when input is tensor([18]) target is 47
when input is tensor([18, 47]) target is 56
when input is tensor([18, 47, 56]) target is 57
when input is tensor([18, 47, 56, 57]) target is 58
when input is tensor([18, 47, 56, 57, 58]) target is 1
when input is tensor([18, 47, 56, 57, 58,  1]) target is 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) target is 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) target is 58


In [ ]:
# batch dimension for parallel processing, process multiple chunks at the same time

torch.manual_seed(1300)
batch_size=4 # how many independent sequences will we process in parallel
block_size=8 # what is the maximum context length for predictions

def get_batch(split):
  # generate a small batch of ata of inputs x and targets y
  data=train_data if split=="train" else val_data
  ix=torch.randint(len(data)-block_size, (batch_size,))
  x=torch.stack([data[i:i+block_size] for i in ix])
  y=torch.stack([data[i+1:i+block_size+1] for i in ix])
  return x,y

xb, yb=get_batch("train")
print("inputs:")
print(xb.shape)
print(xb)
print("targets:")
print(yb.shape)
print(yb)
print("-------")

for b in range(batch_size): # batch dimension
  for t in range(block_size): #time dimension
    context=xb[b, :t+1]
    target=yb[b,t]
    print(f"when input is {context.tolist()} the target is {target}")


inputs:
torch.Size([4, 8])
tensor([[ 0, 57, 53,  1, 58, 46, 39, 58],
        [39, 56, 58, 46,  8,  0, 27,  6],
        [17, 17, 26,  1, 17, 24, 21, 38],
        [43,  1, 57, 43, 52, 58, 43, 52]])
targets:
torch.Size([4, 8])
tensor([[57, 53,  1, 58, 46, 39, 58,  1],
        [56, 58, 46,  8,  0, 27,  6,  1],
        [17, 26,  1, 17, 24, 21, 38, 13],
        [ 1, 57, 43, 52, 58, 43, 52, 41]])
-------
when input is [0] the target is 57
when input is [0, 57] the target is 53
when input is [0, 57, 53] the target is 1
when input is [0, 57, 53, 1] the target is 58
when input is [0, 57, 53, 1, 58] the target is 46
when input is [0, 57, 53, 1, 58, 46] the target is 39
when input is [0, 57, 53, 1, 58, 46, 39] the target is 58
when input is [0, 57, 53, 1, 58, 46, 39, 58] the target is 1
when input is [39] the target is 56
when input is [39, 56] the target is 58
when input is [39, 56, 58] the target is 46
when input is [39, 56, 58, 46] the target is 8
when input is [39, 56, 58, 46, 8] the target is

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1300)

class BigramLanguageModel(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.token_embedding_table=nn.Embedding(vocab_size, vocab_size)

  def forward(self, idx, targets=None):
    logits=self.token_embedding_table(idx) # (B,T,C)
    if targets is None:
      loss=None
    else:
      # pytorch expects()
      B,T,C=logits.shape
      logits=logits.view(B*T, C)
      targets=targets.view(B*T)
      loss=F.cross_entropy(logits, targets)
    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      # get the predictions
      logits,loss=self(idx)
      # focus only on last time step
      logits=logits[:,-1,:] #becomes (B,C)
      # apply softmax to get probabilities
      probs=F.softmax(logits, dim=-1) #(B,C)
      # sample the distribution
      idx_next=torch.multinomial(probs, num_samples=1) #(B,1)
      # append sampled index to the running sequence
      idx=torch.cat((idx, idx_next), dim=1) # (B,T+1)
    return idx

m=BigramLanguageModel(vocab_size)
logits, loss=m(xb,yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.5309, grad_fn=<NllLossBackward0>)


In [ ]:
idx=torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))


m
HRsrBZk:'L!KWcQ
Hc.XpRWAYMr sbzHGRmW:R
mqbzwBtnCPIfZ.X-f,uc-c;iylIjjMpZO?VKN! FzTeOukZBHZhD$ddZWzC


In [ ]:
optimizer=torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
batch_size=32
for steps in range(10000):
  xb,yb=get_batch('train')
  # evaluate the loss
  logits, loss=m(xb, yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

print(loss.item())


2.436859130859375


In [ ]:
idx=torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(idx, max_new_tokens=300)[0].tolist()))


OTAn.
Hothat mout havescor atemar d.
ISOPand d cod toverd t grth a ye windarimat nt tesacelom trf ts HORotursovelens n,Anllod dllomyprayoreild:
ENEdr booom issuld lowown hallir boxcend berd thmito irvaie
Ma pooro;
S:
Cisimatomecel sur's f do. tAy o ie nd acorese br re, fr or atratheathan t ir,
IOUCi


Simplest model because the tokens are not talking to each other

# The mathematical trick in self Attention

In [ ]:
torch.manual_seed(1300)
B,T,C=4,8,2 # batch , time , channels
x=torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
# tokens should talk to past tokens only,context should flow from past to present
# just do average of previous tokens, that would be feature vector, that would summarize the context
# altough lossy but its ok for now

In [ ]:
xbow=torch.zeros((B,T,C))
for b in range(B):
  for t in range(T):
    xprev=x[b,:t+1] #(t,C)
    xbow[b,t]=torch.mean(xprev, 0)

In [ ]:
x[0]

tensor([[-0.2028, -1.4443],
        [-0.5584, -1.0433],
        [ 0.2127,  0.4175],
        [-0.6576,  1.3523],
        [ 0.0918, -1.1213],
        [-1.5167,  1.1398],
        [ 0.4244,  0.8652],
        [ 0.6012,  0.8964]])

In [ ]:
xbow[0]

tensor([[-0.2028, -1.4443],
        [-0.3806, -1.2438],
        [-0.1828, -0.6900],
        [-0.3015, -0.1795],
        [-0.2229, -0.3678],
        [-0.4385, -0.1166],
        [-0.3152,  0.0237],
        [-0.2007,  0.1328]])

In [ ]:
# very inefficient process
# efficient using matrix multiplication
torch.manual_seed(42)
a=torch.ones(3,3)
b=torch.randint(0,10,(3,2)).float()
c= a @ b
print("a=")
print(a)
print("b=")
print(b)
print("c=")
print(c)

a=
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[14., 16.],
        [14., 16.],
        [14., 16.]])


In [ ]:
#the trick here is following
#we get the lower traingular part
torch.tril(torch.ones(3,3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [ ]:
torch.manual_seed(42)
a=torch.tril(torch.ones(3,3))
a=a/torch.sum(a,1, keepdim=True)
b=torch.randint(0,10,(3,2)).float()
c= a @ b
print("a=")
print(a)
print("b=")
print(b)
print("c=")
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [ ]:
# applying this trick above
# version 2
torch.manual_seed(1337)
B,T,C=4,8,2 # batch , time , channels
x=torch.randn(B,T,C)
x.shape


xbow=torch.zeros((B,T,C))
for b in range(B):
  for t in range(T):
    xprev=x[b,:t+1] #(t,C)
    xbow[b,t]=torch.mean(xprev, 0)

print(xbow.shape)

torch.Size([4, 8, 2])


In [ ]:

wei=torch.tril(torch.ones(T,T))
wei=wei/wei.sum(1,keepdim=True)

wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [ ]:
xbow2=wei @ x # (T,T) @ (B,T,C) ----> (B,T,T) @ (B,T,C) ----> (B,T,C) xbow2 will be identical to xbow
torch.allclose(xbow, xbow2)
print(xbow2.shape)

torch.Size([4, 8, 2])


In [ ]:
xbow[0], xbow2[0]

(tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In [ ]:
# version 3
torch.manual_seed(1337)
B,T,C=4,8,2 # batch , time , channels
x=torch.randn(B,T,C)
x.shape


xbow=torch.zeros((B,T,C))
for b in range(B):
  for t in range(T):
    xprev=x[b,:t+1] #(t,C)
    xbow[b,t]=torch.mean(xprev, 0)

tril=torch.tril(torch.ones(T,T))
wei=torch.zeros((T,T))
wei=wei.masked_fill(tril==0, float('-inf'))
print(wei)
wei=F.softmax(wei, dim=-1)
print(wei)
xbow3=wei @ x
xbow[0],xbow2[0],  xbow3[0]

tensor([[0., -inf, -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., -inf, -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., -inf, -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., -inf, -inf, -inf, -inf],
        [0., 0., 0., 0., 0., -inf, -inf, -inf],
        [0., 0., 0., 0., 0., 0., -inf, -inf],
        [0., 0., 0., 0., 0., 0., 0., -inf],
        [0., 0., 0., 0., 0., 0., 0., 0.]])
tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])


(tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]),
 tensor([[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]))

In [ ]:
# v4: Self atttention
torch.manual_seed(1337)
B,T,C=4,8,32 # batch , time , channels
x=torch.randn(B,T,C)

# let's see a single Head perform self attention
head_size=16
key=nn.Linear(C, head_size, bias=False)
query=nn.Linear(C, head_size, bias=False)
value=nn.Linear(C, head_size, bias=False)
k=key(x) #(B,T,16)
q=query(x) # (B,T,16)

wei=q @ k.transpose(-2,-1) #(B,T,16) @ (B,16,T) ----> (B,T,T)
print("Wei kq:",wei)
tril=torch.tril(torch.ones(T,T))
print("tril",tril)
# but i dont want this to be consistent
# wei=torch.zeros((T,T))
wei =wei.masked_fill(tril==0, float('-inf'))
print('wei masked fill', wei)
wei=F.softmax(wei, dim=-1)
print('wei softmax', wei)
v=value(x)
out=wei @ v

print('out', out)
out.shape

Wei kq: tensor([[[-1.7629e+00, -1.3011e+00,  5.6516e-01,  2.1616e+00, -1.0674e+00,
           1.9632e+00,  1.0765e+00, -4.5295e-01],
         [-3.3334e+00, -1.6556e+00,  1.0405e-01,  3.3782e+00, -2.1825e+00,
           1.0415e+00, -5.5714e-02,  2.9273e-01],
         [-1.0226e+00, -1.2606e+00,  7.6228e-02, -3.8125e-01, -9.8430e-01,
          -1.4303e+00,  7.4921e-02, -9.5465e-01],
         [ 7.8359e-01, -8.0143e-01, -3.3680e-01, -8.4963e-01, -5.6023e-01,
          -1.1701e+00, -1.2927e+00, -1.0260e+00],
         [-1.2566e+00,  1.8719e-02, -7.8797e-01, -1.3204e+00,  2.0363e+00,
           8.6381e-01,  3.7188e-01,  9.2577e-01],
         [-3.1262e-01,  2.4152e+00, -1.1058e-01, -9.9305e-01,  3.3449e+00,
          -2.5229e+00,  1.4187e+00,  1.2196e+00],
         [ 1.0876e+00,  1.9652e+00, -2.6213e-01, -3.1579e-01,  6.0905e-01,
           1.2616e+00, -5.4841e-01,  8.0485e-01],
         [-1.8044e+00, -4.1260e-01, -8.3061e-01,  5.8985e-01, -7.9869e-01,
          -5.8560e-01,  6.4332e-01,  6.302

torch.Size([4, 8, 16])

1. attention is communication mechanism.
2. we have 8 nodes , block size is 8,
3. no notion of space, attention simply acts over a set of vectors, that's why we use positional encoding
4. elements across the batch dimension never talk to each other
5. in encoder attention block just delete the single line that does tril, allowing tokens to communicate . above traingular block is used in autoregressive settings, like language modelling.
6. self attention because keys queries and values all come from x, in cross attention, queries come from x but keys and values come from different sources

In [ ]:
# scaled attention , to normalize the keys , queries and values

k=torch.rand(B,T,head_size)
q=torch.rand(B,T,head_size)
wei=q @ k.transpose(-2,-1) *head_size**-0.5

In [ ]:
k.var()

tensor(0.0763)

In [ ]:
q.var()

tensor(0.0808)

In [ ]:
wei.var()

tensor(0.0443)

# Version 2


In [1]:
! wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open("input.txt", "r", encoding= "utf-8") as f:
  text=f.read()

--2025-03-24 14:56:53--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.04s   

2025-03-24 14:56:53 (27.2 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F


# hyperparameters
batch_size=64
block_size=256
max_iters=5000
eval_interval=500
learning_rate=3e-4
device='cuda' if torch.cuda.is_available else 'cpu'
# device='cpu'
eval_iters=200
n_embed=384
n_head=6
n_layer=6
dropout=0.2
# =====================

torch.manual_seed(1337)


with open("input.txt", "r", encoding= "utf-8") as f:
  text=f.read()

# here are all the inique characters that occur inthe text
chars=sorted(list(set(text)))
vocab_size=len(chars)

# Create a mappping from characters to integers
stoi={ ch:i for i, ch in enumerate(chars) }
itos={ i:ch for i, ch in enumerate(chars) }
encode=lambda s: [stoi[c] for c in s] #encoder takes a string, outputs a list of integers
decode=lambda l: ''.join([itos[i] for i in l]) # decoder: takes a list of integers, outputs a string

# Train and test set splits
data=torch.tensor(encode(text), dtype=torch.long)
n=int(0.9*len(data)) # first 90% will be train, rest will be val
train_data=data[:n]
val_data=data[n:]

# data loading
def get_batch(split):
  # generate a small batch of ata of inputs x and targets y
  data=train_data if split=="train" else val_data
  ix=torch.randint(len(data)-block_size, (batch_size,))
  x=torch.stack([data[i:i+block_size] for i in ix])
  y=torch.stack([data[i+1:i+block_size+1] for i in ix])
  x,y=x.to(device), y.to(device)
  return x,y

@torch.no_grad()
def estimate_loss():
  out={}
  model.eval()
  for split in ['train', 'val']:
    losses=torch.zeros(eval_iters)
    for k in range(eval_iters):
      # sample a batch of data
      X,Y=get_batch(split)
      logits, loss=model(X,Y)
      losses[k]=loss.item()
    out[split]=losses.mean()
  model.train()
  return out


class Head(nn.Module):
    """ one head of self-attention """
    def __init__(self,head_size):
      super().__init__()
      self.key=nn.Linear(n_embed, head_size, bias=False)
      self.query=nn.Linear(n_embed, head_size, bias=False)
      self.value=nn.Linear(n_embed, head_size, bias=False)
      self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

      self.dropout=nn.Dropout(dropout)

    def forward(self, x):
      B,T,C=x.shape
      k=self.key(x)
      q=self.query(x)

      # compute attention scores ("affinities")
      wei=q @ k.transpose(-2,-1) * C**-0.5  # (B,T,C) @ (B,C,T) ----> (B,T,T)
      wei=wei.masked_fill(self.tril[:T, :T]==0, float('-inf')) # (B,T,T)
      wei=F.softmax(wei, dim=-1) #(B,T,T)
      wei=self.dropout(wei)
      # perform the weighted aggregation of the values
      v=self.value(x) #(B,T,C)
      out=wei@v #(B,T,T) @ (B,C,T) ---> (B,T,C)
      return out

class MultiHeadAttention(nn.Module):
  """ multiple heads of self attention in parallel """
  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads=nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj=nn.Linear(head_size*num_heads, n_embed)
    self.dropout=nn.Dropout(dropout)

  def forward(self, x):
    out=torch.cat([h(x) for h in self.heads], dim=-1)
    out=self.dropout(self.proj(out))
    return out

class FeedForward(nn.Module):
  """ a simple linear layer followed by a non linearity """

  def __init__(self, n_embed):
    super().__init__()
    self.net=nn.Sequential(
      nn.Linear(n_embed, 4*n_embed),
      nn.ReLU(),
      nn.Linear(4*n_embed, n_embed),
      nn.Dropout(dropout)
    )

  def forward(self, x):
    return self.net(x)

class Block(nn.Module):
  """ Transformer block: communication followed by computation """
  def __init__(self, n_embed, n_head):
    super().__init__()
    head_size=n_embed//n_head
    self.sa=MultiHeadAttention(n_head, n_head)
    self.ffwd=FeedForward(n_embed)
    self.ln1=nn.LayerNorm(n_embed)
    self.ln2=nn.LayerNorm(n_embed)

  def forward(self,x):
    x=x+self.sa(self.ln1(x))
    x=x+self.ffwd(self.ln2(x))
    return x

class BigramLanguageModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.token_embedding_table=nn.Embedding(vocab_size, n_embed)
    self.position_embedding_table=nn.Embedding(block_size, n_embed)

    # self.sa_head=MultiHeadAttention(4,n_embed//4)# i.e. 4 heads of 8 dimensional self-attention
    # self.ffwd=FeedForward(n_embed)

    # replace with blocks
    # self.blocks=nn.Sequential(
    #   Block(n_embed, n_head=4),
    #   Block(n_embed, n_head=4),
    #   Block(n_embed, n_head=4),
    #   nn.LayerNorm(n_embed)
    # )
    self.blocks=nn.Sequential(*[Block(n_embed, n_head=n_head) for _ in range(n_layer)])
    self.lm_head=nn.Linear(n_embed, vocab_size)

  def forward(self, idx, targets=None):
    B,T=idx.shape
    token_emb=self.token_embedding_table(idx) # (B,T,C)
    pos_emb=self.position_embedding_table(torch.arange(T ,device=device)) #(T,C)
    x=token_emb+pos_emb # (B,T,C)

    # x=self.sa_head(x) # apply one head of self attention (B,T,C)
    # x=self.ffwd(x)

    # replace with blocks
    x=self.blocks(x) # (B,T,C)
    logits=self.lm_head(x)# (B,T, vocab_size)
    if targets is None:
      loss=None
    else:
      # pytorch expects()
      B,T,C=logits.shape
      logits=logits.view(B*T, C)
      targets=targets.view(B*T)
      loss=F.cross_entropy(logits, targets)
    return logits, loss

  def generate(self, idx, max_new_tokens):
    for _ in range(max_new_tokens):
      # crop idz to the last block size tokens
      idx_cond=idx[:, -block_size:]
      # get the predictions
      logits,loss=self(idx_cond)
      # focus only on last time step
      logits=logits[:,-1,:] #becomes (B,C)
      # apply softmax to get probabilities
      probs=F.softmax(logits, dim=-1) #(B,C)
      # sample the distribution
      idx_next=torch.multinomial(probs, num_samples=1) #(B,1)
      # append sampled index to the running sequence
      idx=torch.cat((idx, idx_next), dim=1) # (B,T+1)
    return idx


model=BigramLanguageModel()
m=model.to(device)

optimizer=torch.optim.AdamW(m.parameters(), lr=learning_rate)

for iter in range(max_iters):
  if iter%eval_interval ==0:
    losses=estimate_loss()
    print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
  xb,yb=get_batch('train')
  logits,loss=model(xb,yb)
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()


context=torch.zeros((1,1), dtype=torch.long)
print(decode(m.generate(context.to(device), max_new_tokens=300)[0].tolist()))

step 0: train loss 4.6302, val loss 4.6157
step 500: train loss 2.3686, val loss 2.3941
step 1000: train loss 1.9829, val loss 2.0691
step 1500: train loss 1.7553, val loss 1.9047
step 2000: train loss 1.6178, val loss 1.7911
step 2500: train loss 1.5298, val loss 1.7190
step 3000: train loss 1.4702, val loss 1.6691
step 3500: train loss 1.4226, val loss 1.6277
step 4000: train loss 1.3871, val loss 1.6066
step 4500: train loss 1.3558, val loss 1.5821

But with the opperon to steeast-night fomerful.

AEdils:
Away:
Thu'll k not your hellord's wiself goood in
to Have want my inheard than abroad fecure
To you beg the usitomer eremas and you
slewst my recomplexion men in right;
My missed 'I bured your own then hence of it
soverd the telw, I do subto f
